<a href="https://colab.research.google.com/github/rodrigo103/AprendizajeAutomatico/blob/main/TP1_clasificaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Librerías de análisis de datos

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from pandas.api.types import CategoricalDtype

from pandas.plotting import scatter_matrix

from pprint import pprint

In [2]:
# Librerías de Machine Learning

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor
from xgboost import XGBClassifier

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv("drive/MyDrive/AprendizajeAutomatico/OnlineNewsPopularity.csv", sep=', ')

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


Corriendo el siguiente código puedo generar la columna de los target para la claficación.

In [5]:
def get_label(share):
  if share <= 870:
    return 'Very Poor'
  elif share <= 1200:
    return 'Poor'
  elif share <= 1800:
    return 'Average'
  elif share <= 3400:
    return 'Good'
  else:
    return 'Very Good'

df['shares_clasif'] = df['shares'].apply(get_label)
df

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,shares_clasif
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.70,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593,Very Poor
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.70,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711,Very Poor
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,1.00,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500,Average
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.80,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200,Poor
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,1.00,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505,Very Poor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39639,http://mashable.com/2014/12/27/samsung-app-aut...,8.0,11.0,346.0,0.529052,1.0,0.684783,9.0,7.0,1.0,...,0.75,-0.260000,-0.500,-0.125000,0.100000,0.000000,0.400000,0.000000,1800,Average
39640,http://mashable.com/2014/12/27/seth-rogen-jame...,8.0,12.0,328.0,0.696296,1.0,0.885057,9.0,7.0,3.0,...,0.70,-0.211111,-0.400,-0.100000,0.300000,1.000000,0.200000,1.000000,1900,Good
39641,http://mashable.com/2014/12/27/son-pays-off-mo...,8.0,10.0,442.0,0.516355,1.0,0.644128,24.0,1.0,12.0,...,0.50,-0.356439,-0.800,-0.166667,0.454545,0.136364,0.045455,0.136364,1900,Good
39642,http://mashable.com/2014/12/27/ukraine-blasts/,8.0,6.0,682.0,0.539493,1.0,0.692661,10.0,1.0,1.0,...,0.50,-0.205246,-0.500,-0.012500,0.000000,0.000000,0.500000,0.000000,1100,Poor


# Clasificación
Entreno los modelos de clasificación usando la misma configuración que para regresión. Los modelos son los mismos pero en su versión para clasificación.

In [6]:
features = [
    'timedelta', 
    # Words
    'n_tokens_title', 'n_tokens_content', 
    'average_token_length', 
    'n_unique_tokens', 'n_non_stop_words', 'n_non_stop_unique_tokens',
    'num_hrefs', 'num_self_hrefs',
    # Digital Media
    'num_imgs', 'num_videos', 
    # Links
    'self_reference_min_shares', 'self_reference_max_shares', 'self_reference_avg_sharess', 
    # Time
    'weekday_is_monday', 'weekday_is_tuesday', 'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday', 'weekday_is_saturday', 'weekday_is_sunday', 
    'is_weekend', 
    # Keywords
    'num_keywords', 
    'kw_min_min', 'kw_max_min', 'kw_avg_min', 'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg', 'kw_avg_avg', 
    'data_channel_is_lifestyle', 'data_channel_is_entertainment', 'data_channel_is_bus', 'data_channel_is_socmed', 'data_channel_is_tech', 'data_channel_is_world', 
    # Natural Language Processing
    'LDA_00', 'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 
    'global_subjectivity', 'global_sentiment_polarity', 
    'global_rate_positive_words', 'global_rate_negative_words', 
    'rate_positive_words', 'rate_negative_words', 
    'avg_positive_polarity', 'min_positive_polarity', 'max_positive_polarity', 
    'avg_negative_polarity', 'min_negative_polarity', 'max_negative_polarity', 
    'title_subjectivity', 'title_sentiment_polarity', 'abs_title_subjectivity', 'abs_title_sentiment_polarity'
]

# Se separa el dataframe en features (X) y target (y)
X = df[features]
y = df.shares_clasif


# Lo primero que hay que hacer es separar un conjunto de datos para test
trainval_X, test_X, trainval_y, test_y = train_test_split(X, y, random_state = 0, test_size = 0.3)

# Ahora separo el conjunto de datos restante (trainval) en entrenamiento (train) y validación (val)
train_X, val_X, train_y, val_y = train_test_split(trainval_X, trainval_y, random_state = 0)

Relizo un entrenamiento con varios modelos sobre todos los features para establecer una linea de base

In [7]:
scoring = {
    'accuracy'
}

preprocessor = ColumnTransformer(
    [
      ('power_transformer', PowerTransformer(), 
        [
          'num_hrefs',
          'num_self_hrefs',
          'n_tokens_content',
          'self_reference_min_shares', 
          'self_reference_max_shares', 
          'self_reference_avg_sharess',
        ]
      ),
      ('standard_scaler', StandardScaler(), 
        [
          # 'n_tokens_title',
          'global_subjectivity', 
          'global_sentiment_polarity',
          'global_rate_positive_words', 
          'global_rate_negative_words',
          'rate_positive_words',
          'rate_negative_words',
          'avg_positive_polarity'
        ]
      ),
    ], remainder='passthrough')

In [8]:
dt_params = {
  'model__max_leaf_nodes': [None, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 500, 5000],
  'model__random_state': [1]
}

model = DecisionTreeClassifier()

my_pipeline = Pipeline(steps=[ 
    ('standard_scaler', preprocessor),
    ('model', model)])

grid_pipeline = GridSearchCV(my_pipeline, 
                              dt_params, 
                              cv = 5, 
                              scoring = scoring,
                              refit='accuracy',
                              return_train_score=True)

grid_pipeline.fit(train_X, train_y);


# Muestro resultados
print(grid_pipeline.cv_results_.keys())

print(grid_pipeline.cv_results_['param_model__max_leaf_nodes'].data)

print(grid_pipeline.cv_results_["mean_test_accuracy"][grid_pipeline.best_index_])
print(grid_pipeline.cv_results_["std_test_accuracy"][grid_pipeline.best_index_])

print(f'Best parameters {grid_pipeline.best_params_}')
print(f'Best score {grid_pipeline.best_score_}')

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_model__max_leaf_nodes', 'param_model__random_state', 'params', 'split0_test_accuracy', 'split1_test_accuracy', 'split2_test_accuracy', 'split3_test_accuracy', 'split4_test_accuracy', 'mean_test_accuracy', 'std_test_accuracy', 'rank_test_accuracy', 'split0_train_accuracy', 'split1_train_accuracy', 'split2_train_accuracy', 'split3_train_accuracy', 'split4_train_accuracy', 'mean_train_accuracy', 'std_train_accuracy'])
[None 2 3 4 5 6 7 8 9 10 20 30 40 50 500 5000]
0.30948474830844896
0.0038078269036087888
Best parameters {'model__max_leaf_nodes': 30, 'model__random_state': 1}
Best score 0.30948474830844896


In [9]:

# Funcion para comparar distintos modelos
def score_model(model, X_t=train_X, X_v=val_X, y_t=train_y, y_v=val_y):
    model.fit(X_t, y_t)
    preds = model.predict(X_v)
    return accuracy_score(y_v, preds)

models = [
    RandomForestClassifier(random_state=1),
    RandomForestClassifier(n_estimators=50, random_state=1),
    RandomForestClassifier(n_estimators=100, random_state=1),
    # RandomForestClassifier(n_estimators=100, criterion='absolute_error', random_state=1),
    # RandomForestClassifier(n_estimators=200, min_samples_split=20, random_state=1),
    # RandomForestClassifier(n_estimators=100, max_depth=7, random_state=1),
    XGBClassifier(), 
    XGBClassifier(n_estimators=500), 
    XGBClassifier(n_estimators=1000, learning_rate=0.05),
]

for i in range(0, len(models)):
    acc = score_model(models[i])
    print("Model %d Accuracy: %f" % (i+1, acc))

# Pruebo usando early stopping
my_model2 = XGBClassifier(n_estimators=500)
my_model2.fit(train_X, train_y,
              early_stopping_rounds=5, 
             eval_set=[(val_X, val_y)],
             verbose=False)
preds = my_model2.predict(val_X)
print("Accuracy: " + str(accuracy_score(preds, val_y)))

Model 1 Accuracy: 0.254396
Model 2 Accuracy: 0.265494
Model 3 Accuracy: 0.272701
Model 4 Accuracy: 0.283511
Model 5 Accuracy: 0.283511
Model 6 Accuracy: 0.288412
Model 7 Accuracy: 0.294321
Model 8 Accuracy: 0.294321
Model 9 Accuracy: 0.294321
Model 10 Accuracy: 0.294321
Model 11 Accuracy: 0.304987
Model 12 Accuracy: 0.311617
Model 13 Accuracy: 0.304555
Model 14 Accuracy: 0.304987
Model 15 Accuracy: 0.285961
Model 16 Accuracy: 0.256558
Model 17 Accuracy: 0.320986
Model 18 Accuracy: 0.305564
Model 19 Accuracy: 0.320986
Model 20 Accuracy: 0.339291
Model 21 Accuracy: 0.339723


KeyboardInterrupt: ignored

In [ ]:
features = [
    # 'timedelta', 
    # Words
    'n_tokens_title', 'n_tokens_content', 
    # 'average_token_length', 
    # 'n_unique_tokens', 'n_non_stop_words', 'n_non_stop_unique_tokens',
    'num_hrefs', 'num_self_hrefs',
    # Digital Media
    # 'num_imgs', 'num_videos', 
    # Links
    'self_reference_min_shares', 'self_reference_max_shares', 'self_reference_avg_sharess', 
    # Time
    # 'weekday_is_monday', 'weekday_is_tuesday', 'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday', 'weekday_is_saturday', 'weekday_is_sunday', 
    # 'is_weekend', 
    # Keywords
    'num_keywords', 
    'kw_min_min', 'kw_max_min', 'kw_avg_min', 'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg', 'kw_avg_avg', 
    'data_channel_is_lifestyle', 'data_channel_is_entertainment', 'data_channel_is_bus', 'data_channel_is_socmed', 'data_channel_is_tech', 'data_channel_is_world', 
    # Natural Language Processing
    'LDA_00', 'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 
    'global_subjectivity', 'global_sentiment_polarity', 
    'global_rate_positive_words', 'global_rate_negative_words', 
    'rate_positive_words', 'rate_negative_words', 
    'avg_positive_polarity', 'min_positive_polarity', 'max_positive_polarity', 
    'avg_negative_polarity', 'min_negative_polarity', 'max_negative_polarity', 
    'title_subjectivity', 'title_sentiment_polarity', 'abs_title_subjectivity', 'abs_title_sentiment_polarity'
]

# Se separa el dataframe en features (X) y target (y)
X = df[features]
y = df.shares_clasif


# Lo primero que hay que hacer es separar un conjunto de datos para test
trainval_X, test_X, trainval_y, test_y = train_test_split(X, y, random_state = 0, test_size = 0.3)

# Ahora separo el conjunto de datos restante (trainval) en entrenamiento (train) y validación (val)
train_X, val_X, train_y, val_y = train_test_split(trainval_X, trainval_y, random_state = 0)

La mejor precisión observada es de 0.318 para el modelo 20, que es el XGBClassifier() sin parámetros. Esta precisión supera la linea de base aleatoria de 0.2.

Para ese modelo vamos a calcular y mostrar las matrices de confusion.

In [ ]:
my_model2 = XGBClassifier()
my_model2.fit(train_X, train_y)
preds = my_model2.predict(val_X)
print("Accuracy: " + str(accuracy_score(preds, val_y)))


confusion = confusion_matrix(val_y, preds)
print(f"Confusion matrix:\n{confusion}")

print(f"\nNormalized confusion matrix:")
for row in confusion:
    print(row / row.sum())


print(f"\nProbs:")
probs = my_model2.predict_proba(val_X)
print(probs)


print(f"\nClassification report:")
report = classification_report(val_y, preds)
print(report)

Accuracy: 0.31621430733528405
Confusion matrix:
[[147 349 528 496 369]
 [159 415 372 574 297]
 [137 243 753 441 666]
 [107 355 291 925 268]
 [ 84 127 602 312 894]]

Normalized confusion matrix:
[0.07781895 0.18475384 0.27951297 0.26257279 0.19534145]
[0.08750688 0.22839846 0.20473308 0.31590534 0.16345625]
[0.06116071 0.10848214 0.33616071 0.196875   0.29732143]
[0.05498458 0.18242549 0.14953751 0.47533402 0.1377184 ]
[0.04160475 0.06290243 0.29816741 0.15453195 0.44279346]

Probs:
[[0.20386672 0.10750477 0.34347168 0.0690873  0.27606955]
 [0.17825504 0.23596886 0.2241451  0.15964583 0.20198514]
 [0.28839678 0.27310723 0.1894298  0.15224674 0.09681943]
 ...
 [0.20587832 0.10839653 0.3830459  0.06823114 0.23444806]
 [0.2515199  0.3312027  0.13662685 0.1909773  0.08967327]
 [0.172007   0.12765108 0.2947282  0.10166623 0.3039475 ]]

Classification report:
              precision    recall  f1-score   support

     Average       0.23      0.08      0.12      1889
        Good       0.28   